In [4]:
#import dependencies
import pandas as pd
from pathlib import Path

In [5]:
#read in homeless csv
home_df = pd.read_csv(Path('Resources/clean_homeless.csv'))
home_df.head()

,State_Year,Year,State,Measures,Count
0,2007_AK,2007,AK,Chronically Homeless Individuals,224
1,2007_AK,2007,AK,Homeless Individuals,696
2,2007_AK,2007,AK,Homeless People in Families,278
3,2007_AK,2007,AK,Sheltered Chronically Homeless Individuals,187
4,2007_AK,2007,AK,Sheltered Homeless,842


In [6]:
#check data types
home_df.dtypes

State_Year    object
Year           int64
State         object
Measures      object
Count          int64
dtype: object

In [7]:
#remove commas from Count 
#home_df['Count'] = home_df['Count'].str.replace(',','')

In [8]:
#convert Count to int
home_df['Count'] = pd.to_numeric(home_df['Count'])
home_df.dtypes

State_Year    object
Year           int64
State         object
Measures      object
Count          int64
dtype: object

In [9]:
#get homeless types
homeless_measure = home_df['Measures'].unique()
homeless_measure

array(['Chronically Homeless Individuals', 'Homeless Individuals',
       'Homeless People in Families',
       'Sheltered Chronically Homeless Individuals', 'Sheltered Homeless',
       'Sheltered Homeless Individuals',
       'Sheltered Homeless People in Families', 'Total Homeless',
       'Unsheltered Chronically Homeless Individuals',
       'Unsheltered Homeless', 'Unsheltered Homeless Individuals',
       'Unsheltered Homeless People in Families', 'Chronically Homeless',
       'Chronically Homeless People in Families', 'Homeless Veterans',
       'Sheltered Chronically Homeless',
       'Sheltered Chronically Homeless People in Families',
       'Sheltered Homeless Veterans', 'Unsheltered Chronically Homeless',
       'Unsheltered Chronically Homeless People in Families',
       'Unsheltered Homeless Veterans', 'Children of Parenting Youth',
       'Homeless Unaccompanied Children (Under 18)',
       'Homeless Unaccompanied Young Adults (Age 18-24)',
       'Homeless Unaccompan

In [10]:
#bin sheltered vs nonsheltered
grouping_lists = [['Sheltered Chronically Homeless Individuals','Sheltered Homeless','Sheltered Homeless Individuals',
                  'Sheltered Homeless People in Families','Sheltered Chronically Homeless','Sheltered Chronically Homeless People in Families',
                  'Sheltered Homeless Veterans','Sheltered Children of Parenting Youth','Sheltered Homeless Unaccompanied Children (Under 18)',
                  'Sheltered Homeless Unaccompanied Young Adults (Age 18-24)','Sheltered Homeless Unaccompanied Youth (Under 25)','Sheltered Parenting Youth (Under 25)',
                  'Sheltered Parenting Youth Age 18-24','Sheltered Parenting Youth Under 18',],['Unsheltered Chronically Homeless Individuals',
                  'Unsheltered Homeless', 'Unsheltered Homeless Individuals','Unsheltered Homeless People in Families',
                  'Unsheltered Chronically Homeless','Unsheltered Chronically Homeless People in Families','Unsheltered Homeless Veterans',
                  'Unsheltered Children of Parenting Youth','Unsheltered Homeless Unaccompanied Children (Under 18)',
                  'Unsheltered Homeless Unaccompanied Young Adults (Age 18-24)','Unsheltered Homeless Unaccompanied Youth (Under 25)','Unsheltered Parenting Youth (Under 25)',
                  'Unsheltered Parenting Youth Age 18-24','Unsheltered Parenting Youth Under 18'], ['Chronically Homeless Individuals', 'Homeless Individuals',
                  'Homeless People in Families','Total Homeless','Chronically Homeless','Chronically Homeless People in Families', 'Homeless Veterans',
                  'Children of Parenting Youth','Homeless Unaccompanied Children (Under 18)','Homeless Unaccompanied Young Adults (Age 18-24)','Homeless Unaccompanied Youth (Under 25)',
                  'Parenting Youth (Under 25)', 'Parenting Youth Age 18-24','Parenting Youth Under 18',]]
group_names = ['Sheltered', 'Unsheltered','Other']

groups = (pd.DataFrame({'Groups':group_names, 'Measures': grouping_lists})
         .explode('Measures')
         .reset_index(drop=True))

home_df = home_df.merge(groups, on='Measures', how='left')
home_df.head()

,State_Year,Year,State,Measures,Count,Groups
0,2007_AK,2007,AK,Chronically Homeless Individuals,224,Other
1,2007_AK,2007,AK,Homeless Individuals,696,Other
2,2007_AK,2007,AK,Homeless People in Families,278,Other
3,2007_AK,2007,AK,Sheltered Chronically Homeless Individuals,187,Sheltered
4,2007_AK,2007,AK,Sheltered Homeless,842,Sheltered


In [11]:
#drop Measures col
home_df = home_df.drop(columns=['Measures'])
home_df.head()

,State_Year,Year,State,Count,Groups
0,2007_AK,2007,AK,224,Other
1,2007_AK,2007,AK,696,Other
2,2007_AK,2007,AK,278,Other
3,2007_AK,2007,AK,187,Sheltered
4,2007_AK,2007,AK,842,Sheltered


In [12]:
#groupby counts
group_home_df = home_df.groupby(['State_Year','Year','State','Groups'], as_index=False).sum()
group_home_df.head()

,State_Year,Year,State,Groups,Count
0,2007_AK,2007,AK,Other,3562
1,2007_AK,2007,AK,Sheltered,2995
2,2007_AK,2007,AK,Unsheltered,567
3,2007_AL,2007,AL,Other,11897
4,2007_AL,2007,AL,Sheltered,8075


In [13]:
#flattened table for groups and counts
flattened_df = group_home_df.pivot(columns='Groups',values='Count')
flattened_df.head()

Groups,Other,Sheltered,Unsheltered
0,3562.0,NaN,NaN
1,NaN,2995.0,NaN
2,NaN,NaN,567.0
3,11897.0,NaN,NaN
4,NaN,8075.0,NaN


In [14]:
#remove Groups index name
flattened_df.columns.name=''
flattened_df.head()

,Other,Sheltered,Unsheltered
0,3562.0,NaN,NaN
1,NaN,2995.0,NaN
2,NaN,NaN,567.0
3,11897.0,NaN,NaN
4,NaN,8075.0,NaN


In [15]:
#merge flattend df to main df
merged_flat_df = group_home_df.join(flattened_df,how='inner')
merged_flat_df.head()

,State_Year,Year,State,Groups,Count,Other,Sheltered,Unsheltered
0,2007_AK,2007,AK,Other,3562,3562.0,NaN,NaN
1,2007_AK,2007,AK,Sheltered,2995,NaN,2995.0,NaN
2,2007_AK,2007,AK,Unsheltered,567,NaN,NaN,567.0
3,2007_AL,2007,AL,Other,11897,11897.0,NaN,NaN
4,2007_AL,2007,AL,Sheltered,8075,NaN,8075.0,NaN


In [16]:
#drop columns
merged_flat_df = merged_flat_df.drop(columns=['Count','Groups'])
merged_flat_df.head()

,State_Year,Year,State,Other,Sheltered,Unsheltered
0,2007_AK,2007,AK,3562.0,NaN,NaN
1,2007_AK,2007,AK,NaN,2995.0,NaN
2,2007_AK,2007,AK,NaN,NaN,567.0
3,2007_AL,2007,AL,11897.0,NaN,NaN
4,2007_AL,2007,AL,NaN,8075.0,NaN


In [17]:
#fill Nan
merged_flat_df = merged_flat_df.fillna(0)

In [18]:
#groupby 
grouped_flat_df = merged_flat_df.groupby(['State_Year','Year','State'], as_index=False).sum()
grouped_flat_df.head()

,State_Year,Year,State,Other,Sheltered,Unsheltered
0,2007_AK,2007,AK,3562.0,2995.0,567.0
1,2007_AL,2007,AL,11897.0,8075.0,3822.0
2,2007_AR,2007,AR,8524.0,4979.0,3545.0
3,2007_AZ,2007,AZ,32096.0,17886.0,14210.0
4,2007_CA,2007,CA,318313.0,102972.0,215341.0


In [19]:
#convert floats to int
grouped_flat_df['Other'] = grouped_flat_df['Other'].astype(int)
grouped_flat_df['Sheltered'] = grouped_flat_df['Sheltered'].astype(int)
grouped_flat_df['Unsheltered'] = grouped_flat_df['Unsheltered'].astype(int)
grouped_flat_df.dtypes

State_Year     object
Year            int64
State          object
Other           int64
Sheltered       int64
Unsheltered     int64
dtype: object

In [20]:
#rename columns and reorder
final_home_df = grouped_flat_df.rename(columns={'Other': 'Other_Cnt','Sheltered':'Sheltered_Cnt','Unsheltered':'Unsheltered_Cnt'})
final_home_df = final_home_df[['State_Year','Year','State','Sheltered_Cnt','Unsheltered_Cnt','Other_Cnt']]
final_home_df.head()

,State_Year,Year,State,Sheltered_Cnt,Unsheltered_Cnt,Other_Cnt
0,2007_AK,2007,AK,2995,567,3562
1,2007_AL,2007,AL,8075,3822,11897
2,2007_AR,2007,AR,4979,3545,8524
3,2007_AZ,2007,AZ,17886,14210,32096
4,2007_CA,2007,CA,102972,215341,318313


In [21]:
#export preprocessed data
final_home_df.to_csv('Resources/processed_homeless.csv',index=False)

In [22]:
#read in education csv
edu_df = pd.read_csv(Path('Resources/clean_education.csv'))
edu_df.head()

,State_Year,STATE,YEAR,TOTAL_REVENUE,TOTAL_EXPENDITURE,GRADES_9_12_G,GRADES_ALL_G
0,1992_AL,AL,1992,2678885.0,2653798.0,0.0,731634.0
1,1992_AK,AK,1992,1049591.0,972488.0,0.0,122487.0
2,1992_AZ,AZ,1992,3258079.0,3401580.0,0.0,673477.0
3,1992_AR,AR,1992,1711959.0,1743022.0,0.0,441490.0
4,1992_CA,CA,1992,26260025.0,27138832.0,0.0,5254844.0


In [23]:
edu_df['TOTAL_REVENUE'] = edu_df['TOTAL_REVENUE'].astype(int)
edu_df['TOTAL_EXPENDITURE'] = edu_df['TOTAL_EXPENDITURE'].astype(int)
edu_df['GRADES_9_12_G'] = edu_df['GRADES_9_12_G'].astype(int)
edu_df['GRADES_ALL_G'] = edu_df['GRADES_ALL_G'].astype(int)
edu_df.head()

,State_Year,STATE,YEAR,TOTAL_REVENUE,TOTAL_EXPENDITURE,GRADES_9_12_G,GRADES_ALL_G
0,1992_AL,AL,1992,2678885,2653798,0,731634
1,1992_AK,AK,1992,1049591,972488,0,122487
2,1992_AZ,AZ,1992,3258079,3401580,0,673477
3,1992_AR,AR,1992,1711959,1743022,0,441490
4,1992_CA,CA,1992,26260025,27138832,0,5254844


In [24]:
#export preprocessed data
edu_df.to_csv('Resources/processed_education.csv',index=False)

In [28]:
#merge datasets
merged_df = final_home_df.merge(edu_df, on='State_Year',how='inner')
merged_df.head()

,State_Year,Year,State,Sheltered_Cnt,Unsheltered_Cnt,Other_Cnt,STATE,YEAR,TOTAL_REVENUE,TOTAL_EXPENDITURE,GRADES_9_12_G,GRADES_ALL_G
0,2007_AK,2007,AK,2995,567,3562,AK,2007,1800616,1938755,42049,131029
1,2007_AL,2007,AL,8075,3822,11897,AL,2007,7069040,7196459,216941,742919
2,2007_AR,2007,AR,4979,3545,8524,AR,2007,4415981,4779308,138921,479016
3,2007_AZ,2007,AZ,17886,14210,32096,AZ,2007,8724434,8709531,316376,1087447
4,2007_CA,2007,CA,102972,215341,318313,CA,2007,72516936,73225422,2011865,6343471


In [29]:
merged_df = merged_df.drop(columns=['STATE','YEAR','State_Year'])
merged_df.head()

,Year,State,Sheltered_Cnt,Unsheltered_Cnt,Other_Cnt,TOTAL_REVENUE,TOTAL_EXPENDITURE,GRADES_9_12_G,GRADES_ALL_G
0,2007,AK,2995,567,3562,1800616,1938755,42049,131029
1,2007,AL,8075,3822,11897,7069040,7196459,216941,742919
2,2007,AR,4979,3545,8524,4415981,4779308,138921,479016
3,2007,AZ,17886,14210,32096,8724434,8709531,316376,1087447
4,2007,CA,102972,215341,318313,72516936,73225422,2011865,6343471


In [30]:
merged_df = merged_df[['Year','State','TOTAL_REVENUE','TOTAL_EXPENDITURE','GRADES_9_12_G',
             'GRADES_ALL_G','Sheltered_Cnt','Unsheltered_Cnt','Other_Cnt']]
merged_df.head()

,Year,State,TOTAL_REVENUE,TOTAL_EXPENDITURE,GRADES_9_12_G,GRADES_ALL_G,Sheltered_Cnt,Unsheltered_Cnt,Other_Cnt
0,2007,AK,1800616,1938755,42049,131029,2995,567,3562
1,2007,AL,7069040,7196459,216941,742919,8075,3822,11897
2,2007,AR,4415981,4779308,138921,479016,4979,3545,8524
3,2007,AZ,8724434,8709531,316376,1087447,17886,14210,32096
4,2007,CA,72516936,73225422,2011865,6343471,102972,215341,318313


In [31]:
#encode State
merged_encoded_df = pd.get_dummies(merged_df, columns=['State'])
merged_encoded_df

,Year,TOTAL_REVENUE,TOTAL_EXPENDITURE,GRADES_9_12_G,GRADES_ALL_G,Sheltered_Cnt,Unsheltered_Cnt,Other_Cnt,State_AK,State_AL,...,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY
0,2007,1800616,1938755,42049,131029,2995,567,3562,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2007,7069040,7196459,216941,742919,8075,3822,11897,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2007,4415981,4779308,138921,479016,4979,3545,8524,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2007,8724434,8709531,316376,1087447,17886,14210,32096,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2007,72516936,73225422,2011865,6343471,102972,215341,318313,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,2016,2112365,2082696,25573,88428,2437,444,2881,0,0,...,0,0,0,0,0,1,0,0,0,0
506,2016,14964364,15253296,339349,1101711,30103,21828,51931,0,0,...,0,0,0,0,0,0,1,0,0,0
507,2016,11697466,11787535,262681,864432,12723,1105,13828,0,0,...,0,0,0,0,0,0,0,1,0,0
508,2016,3391579,3366566,79442,273855,3103,407,3510,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
#scale columns (standardscaler)
#def scale(val):
#    return val/100

#merged_encoded_df['TOTAL_REVENUE'] = merged_encoded_df['TOTAL_REVENUE'].apply(scale)
#merged_encoded_df['TOTAL_EXPENDITURE'] = merged_encoded_df['TOTAL_EXPENDITURE'].apply(scale)
#merged_encoded_df['GRADES_9_12_G'] = merged_encoded_df['GRADES_9_12_G'].apply(scale)
#merged_encoded_df['GRADES_ALL_G'] = merged_encoded_df['GRADES_ALL_G'].apply(scale)
#merged_encoded_df

In [32]:
#export preprocessed data
merged_encoded_df.to_csv('Resources/homeless_edu.csv',index=False)